# Inference notebook (only for submissions)

In [ ]:
!pip install ../input/efficientnet-pytorch
!pip install "../input/pretrained-models/pretrained-models.pytorch-master"


In [ ]:
from fastai.vision.all import *
from efficientnet_pytorch import EfficientNet
import albumentations
from albumentations.pytorch import ToTensorV2
#import torchvision.models as pretrainedmodels

# Submission setup

Creating paths and CFG for use

In [ ]:
class SubmissionConfig:
    n_tta       = 1
    beta        = 0.25
    models      = ['model-f0.pkl', 'model-f1.pkl', 'model-f2.pkl', 'model-f3.pkl', 'model-f4.pkl']
    
cfg = SubmissionConfig()

In [ ]:
# this is only for submissions
path_str = '../input/cassava-leaf-disease-classification'

submission_df = pd.read_csv(f'{path_str}/sample_submission.csv')

models_path = '../input/effnetmodels/'
test_images_path = f'{path_str}/test_images/'
test_data_path = submission_df['image_id'].apply(lambda x: test_images_path+x)

# ALL COMBINED[](http://)

In [ ]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from albumentations.pytorch import ToTensorV2


all_album = [albumentations.Resize(512,512),albumentations.Transpose(p=1.),albumentations.HorizontalFlip(p=1.),
            albumentations.VerticalFlip(p=1.),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=1.
            ), albumentations.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),]

In [ ]:
len(all_album)-1

## For Efficient Net

In [ ]:
final_predictions = 0
for i in range(6):
    class AlbumentationsTransform(RandTransform):
        split_idx,order = None, 2
    
        def __init__(self, train_aug, valid_aug): 
            store_attr()
    
        def before_call(self, b, split_idx):
            self.idx = split_idx
    
        def encodes(self, img: PILImage):
            if self.idx == 0:
                aug_img = self.train_aug(image=np.array(img))['image']
            else:
                aug_img = self.valid_aug(image=np.array(img))['image']
            return PILImage.create(aug_img)
    
    def get_train_aug(size): 
        return albumentations.Compose([
                all_album[0],all_album[i+1]
    ], p=1.)
    def get_valid_aug(size): 
        return albumentations.Compose([
            albumentations.Resize(size, size),
            albumentations.CenterCrop(size, size, p=1.),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
    ], p=1.), 


    def get_x(row): return images_path/row['image_id']
    def get_y(row): return row['label']
    
    
    predictions = 0

    for model in cfg.models:
        learn = load_learner(Path(models_path + model), cpu=False).to_fp16()
        learn_tst_dl = learn.dls.test_dl(test_data_path)

        learn.cbs.pop(3) # remove wandb
        learn.cbs.pop(3) # remove cutmix
        learn.cbs.pop(3) # remove savemodel callback
    
        learn_predictions = learn.tta(dl=learn_tst_dl, n=cfg.n_tta, beta=cfg.beta)
        predictions += learn_predictions[0]

    

    predictions = predictions / len(cfg.models)
    final_predictions += predictions
    
final_predictions = final_predictions/(len(all_album)-1)

    
    

In [ ]:
path_str = '../input/cassava-leaf-disease-classification'

models_path = '../input/resnextmodels/'
test_images_path = f'{path_str}/test_images/'
#test_data_path = submission_df['image_id'].apply(lambda x: test_images_path+x)

## For Resext


In [ ]:
final_predictions_resnext = 0
for i in range(4):
    class AlbumentationsTransform(RandTransform):
        split_idx,order = None, 2
    
        def __init__(self, train_aug, valid_aug): 
            store_attr()
    
        def before_call(self, b, split_idx):
            self.idx = split_idx
    
        def encodes(self, img: PILImage):
            if self.idx == 0:
                aug_img = self.train_aug(image=np.array(img))['image']
            else:
                aug_img = self.valid_aug(image=np.array(img))['image']
            return PILImage.create(aug_img)
    
    def get_train_aug(size): 
        return albumentations.Compose([
                all_album[0],all_album[i+1]
    ], p=1.)
    def get_valid_aug(size): 
        return albumentations.Compose([
            albumentations.Resize(size, size),
            albumentations.CenterCrop(size, size, p=1.),
    ], p=1.)


    def get_x(row): return images_path/row['image_id']
    def get_y(row): return row['label']
    
    
    predictions = 0

    for model in cfg.models:
        learn = load_learner(Path(models_path + model), cpu=False).to_fp16()
        learn_tst_dl = learn.dls.test_dl(test_data_path)

        learn.cbs.pop(3) # remove wandb
        learn.cbs.pop(3) # remove cutmix
        learn.cbs.pop(3) # remove savemodel callback
    
        learn_predictions = learn.tta(dl=learn_tst_dl, n=cfg.n_tta, beta=cfg.beta)
        predictions += learn_predictions[0]

    

    predictions = predictions / len(cfg.models)
    final_predictions_resnext += predictions
    
final_predictions_resnext = final_predictions_resnext/(len(all_album)-1)
    

In [ ]:
final_final_predictions = 0.5*final_predictions + 0.5*final_predictions_resnext



# Augmentations for inference 

In [ ]:
"""
class AlbumentationsTransform(RandTransform):
    split_idx,order = None, 2
    
    def __init__(self, train_aug, valid_aug): 
        store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)
    
def get_train_aug(size): 
    return albumentations.Compose([
            albumentations.RandomResizedCrop(size,size),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
], p=1.)
def get_valid_aug(size): 
    return albumentations.Compose([
        albumentations.Resize(size, size),
        albumentations.CenterCrop(size, size, p=1.),
], p=1.)


def get_x(row): return images_path/row['image_id']
def get_y(row): return row['label']
"""

# Prediction loop

In [ ]:
"""
predictions = 0

for model in cfg.models:
    learn = load_learner(Path(models_path + model), cpu=False).to_fp16()
    learn_tst_dl = learn.dls.test_dl(test_data_path)

    learn.cbs.pop(3) # remove wandb
    learn.cbs.pop(3) # remove cutmix
    learn.cbs.pop(3) # remove savemodel callback
    
    learn_predictions = learn.tta(dl=learn_tst_dl, n=cfg.n_tta, beta=cfg.beta)
    predictions += learn_predictions[0]

    
# average the predictions
predictions = predictions / len(cfg.models)
"""

# Submit

In [ ]:
submission_df['label'] = np.argmax(final_final_predictions, axis=1)
submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission_df